In [2]:
BASE_DIR = '/Users/amarmandal/Documents/coding/gpt/'

In [18]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import numpy as np
device = 'mps' if torch.backends.mps.is_available() else 'cpu'
print("Device:", device)
block_size = 8
batch_size = 4

Device: mps


## Load the data

In [19]:
with open(BASE_DIR + "data/wizard_of_oz.txt", 'r', encoding='utf-8') as f:
    text = f.read()

chars = sorted(set(text))
print(chars)
vocab_size = len(chars)

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']


## encode decode characters

In [49]:
string_to_int = {ch:i for i, ch in enumerate(chars)}
int_to_string = {i:ch for i, ch in enumerate(chars)}
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

encoded_hello = encode("!r")
decoded_hello = decode(encoded_hello)
encoded_hello, decoded_hello

([2, 71], '!r')

## Convert text to tensors

In [21]:
data = torch.tensor(encode(text), dtype=torch.long)
print(data[:100])

tensor([80,  1,  1, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44, 32, 29,
         1, 47, 33, 50, 25, 42, 28,  1, 33, 38,  1, 39, 50,  0,  0,  1,  1, 26,
        49,  0,  0,  1,  1, 36, 11,  1, 30, 42, 25, 38, 35,  1, 26, 25, 45, 37,
         0,  0,  1,  1, 25, 45, 44, 32, 39, 42,  1, 39, 30,  1, 44, 32, 29,  1,
        47, 33, 50, 25, 42, 28,  1, 39, 30,  1, 39, 50,  9,  1, 44, 32, 29,  1,
        36, 25, 38, 28,  1, 39, 30,  1, 39, 50])


## Train Validation split

In [22]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    print("index:", ix)
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x,y

x, y = get_batch('train')
print('inputs:')
print(x)
print('targets:')
print(y)

index: tensor([ 97321, 100135,  69381, 173742])
inputs:
tensor([[ 0,  0,  3, 39, 61,  9,  1, 67],
        [61, 58,  1, 64, 62, 67, 57,  1],
        [54, 72,  1, 73, 62, 66, 58,  1],
        [61, 58, 67,  1, 61, 58,  1, 76]], device='mps:0')
targets:
tensor([[ 0,  3, 39, 61,  9,  1, 67, 68],
        [58,  1, 64, 62, 67, 57,  1, 55],
        [72,  1, 73, 62, 66, 58,  1, 59],
        [58, 67,  1, 61, 58,  1, 76, 54]], device='mps:0')


## create input and target label

In [23]:
x = train_data[:block_size]
y = train_data[1:block_size+1]

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print("when input is", context, "target is", target)

when input is tensor([80]) target is tensor(1)
when input is tensor([80,  1]) target is tensor(1)
when input is tensor([80,  1,  1]) target is tensor(28)
when input is tensor([80,  1,  1, 28]) target is tensor(39)
when input is tensor([80,  1,  1, 28, 39]) target is tensor(42)
when input is tensor([80,  1,  1, 28, 39, 42]) target is tensor(39)
when input is tensor([80,  1,  1, 28, 39, 42, 39]) target is tensor(44)
when input is tensor([80,  1,  1, 28, 39, 42, 39, 44]) target is tensor(32)


In [24]:
train_data[:30]

tensor([80,  1,  1, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44, 32, 29,
         1, 47, 33, 50, 25, 42, 28,  1, 33, 38,  1, 39])

## create architecture

In [52]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)

        if targets is None:
            loss = None
        else:
            B,T,C = logits.shape
            logits = torch.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss


    def generate(self, index, max_new_tokens):
        # index is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self.forward(index)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax function to get probabilites
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            index = torch.cat((index, index_next), dim=-1) # (B, T+1)
        return index


model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.tensor([[1, 2], [4, 5]], dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)

 !Vpgz_G7-gmq;Yobj-
p(vcdDx*h0lSCrdstc1nQ;GdhAnG5XC F0zMQ&v;5X0s7a?TQH0s0.kTs-xe[w,p-:cUfujh2NeRO)4[JYzBcEBd_hwXhPoCASy]jZ[htyDux(&Be!G3y"AZJo[;.ELzO_)[HK!sVbqiQ-RzxrJ1L﻿07kNUovzaZ;";,3[Etei? Zl﻿"T7PaBwPacce 4!8P"bpJNY7Xjp"E3ZnZ;vMg"fO*h1 N]0BdcGzOziX'D::zgA[act;XK]-R]ft7
)SC﻿OE38!ZZv9;YfzO&cWna9z_19GDXa6PucVpMg1DlPubR9-EnR7PQI27L_aCHK:fyYf7]PA16T8b6i0ML
c;4﻿9rdiudCuFZExESKnQH,oHcN8BdZDlowCujI!i0HmeXGhLe,[ko.jcVCO&f7PJ)456vrR_zU[
UMv2"kIXJ﻿hB*IB0C
(O&BM﻿wA0V;'pg;'b"CE0o3k&KpdZ'iCNge(5,eaZE]- YEp";
